In [1]:
import pandas as pd
import numpy as np

In [2]:
prop = pd.read_csv('properties_2017.csv', usecols=['parcelid',
                                                        'latitude',
                                                        'longitude',
                                                         ])
crime = pd.read_csv('crime.csv', usecols=['Location ',])

In [3]:
geo_crime = crime['Location '].str.strip('()').str.split(', ', expand=True).rename(columns={0:'Latitude', 1:'Longitude'})
geo_crime.columns = ['Latitude','Longitude']
geo_crime = geo_crime.apply(pd.to_numeric)
geo_crime = geo_crime.dropna()

In [4]:
lat_max = geo_crime.loc[geo_crime['Latitude'].idxmax()]['Latitude']
lat_min = geo_crime.loc[geo_crime['Latitude'].idxmin()]['Latitude']
lon_max = geo_crime.loc[geo_crime['Longitude'].idxmax()]['Longitude']
lon_min = geo_crime.loc[geo_crime['Longitude'].idxmin()]['Longitude']

In [5]:
# We want to devide the whole area into 10000 blocks where each block include around 160 coordinates of (lat,lan).
# Number of coordinates in government crime dataset : 1600000

offset_lat = (lat_max - lat_min) / 100
offset_lon = (lon_max - lon_min) / 100
Blocks = pd.DataFrame(columns=['latitude_start','latitude_end','longitude_start','longitude_end','rate'])


In [6]:
lat = pd.DataFrame({'A': np.linspace(lat_min, lat_max, 100)})
lon = pd.DataFrame({'B': np.linspace(lon_min, lon_max, 100)})
lat.columns = ['lat']
lon.columns = ['lon']
rate = np.zeros((100,100))

In [7]:
for index, row in geo_crime.iterrows():
    a = row['Latitude']
    b = row['Longitude']
    for i in range(1,100):
        if lat.loc[i-1,'lat'] <= a and a <= lat.loc[i,'lat']:
            for j in range(1,100):
                if lon.loc[j-1,'lon'] <= b and b <= lon.loc[j,'lon']:
                    rate[i,j] = rate[i,j]  + 1
                    break

In [11]:
rate.sum()

1602888.0

In [12]:
prop = prop.dropna()
prop['latitude'] = prop['latitude'] / 1000000
prop['longitude'] = prop['longitude'] / 1000000
prop['rate'] = 0

In [15]:
count = 0
for index, row in prop.iterrows():
    a = row['latitude']
    b = row['longitude']
    for i in range(1,10):
        if lat.loc[i-1,'lat'] < a and a < lat.loc[i,'lat']:
            for j in range(1,10):
                if lon.loc[j-1,'lon'] < b and b < lon.loc[j,'lon']:
                    row['rate'] = rate[i,j]
                    count = count + 1
                    break

In [18]:
prop['rate'].sum()
count

0

In [22]:
print(lat.tail(5))
print(prop.head(5))

          lat
95  33.385015
96  33.736436
97  34.087858
98  34.439279
99  34.790700
   parcelid   latitude   longitude  rate
0  10754147  34.144442 -118.654084     0
1  10759547  34.140430 -118.625364     0
2  10843547  33.989359 -118.394633     0
3  10859147  34.148863 -118.437206     0
4  10879947  34.194168 -118.385816     0


In [87]:
# lat = lat_min
# lon = lon_min

# Blocks.loc[index] = pd.Series({'latitude_start':lat, 'latitude_end': (lat + offset_lat), 'longitude_start':lon, 'longitude_end':(lon + offset_lon)})

# for i in range(100):
#     lat += offset_lat
#     for j in range(100):
#         lon += offset_lon
#         Blocks.loc[index+1,'latitude_end'] = lat
#         Blocks.loc[index+1,'longitude_end'] = lon
#         Blocks.loc[index+1,'latitude_start'] = Blocks.loc[index ,'latitude_end'] + 0.001
#         Blocks.loc[index+1,'longitude_start'] =  Blocks.loc[index ,'longitude_end'] + 0.001
#         index += 1